#### 0. Sets all the pathways 

In [1]:
import os

## Universal paths
iupred_path = "/home/pythagoras/Programs/iupred3"
psipred_path = "/home/pythagoras/Programs/psipred"
NCBI_protein_database = "/home/pythagoras/Documents/PhD/PSSMSearch/protein.fasta"
bait_sequence = (
    "MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVGDEAQSKRGILTLKYPIEHGIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETFNTPAMYVAIQAVLSLYASGRTTGIVMDSGDGVTHTVPIYEGYALPHAILRLDLAGRDLTDYLMKILTERGYSFTTTAEREIVRDIKEKLCYVALDFEQEMATAASSSSLEKSYELPDGQVITIGNERFRCPEALFQPSFLGMESCGIHETTFNSIMKCDVDIRKDLYANTVLSGGTTMYPGIADRMQKEITALAPSTMKIKIIAPPERKYSVWIGGSILASLSTFQQMWISKQEYDESGPSIVHRKCF"
)
uniref90_path = "/home/pythagoras/Programs/Uniref/Uniref90/uniref90.fasta"
reformat_path = "/home/pythagoras/Programs/hhsuite/hh-suite/build/scripts"

def setup_environment():
    """Prompts for project name and sets up the folder structure."""
    # Prompt for project name
    project_name = input("Enter the project name: ").strip()
    
    # Define base directory
    base_dir = os.path.join(os.getcwd(), project_name)
    
    # Create base directory if it doesn't exist
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        print(f"Project folder '{base_dir}' created.")
    else:
        print(f"Project folder '{base_dir}' already exists.")
    
    # Define and create subdirectories
    subdirs = {
        "input": os.path.join(base_dir, "Input"),
        "output_fasta": os.path.join(base_dir, "Output/Fasta"),
        "output_a3m": os.path.join(base_dir, "Output/a3m"),
        "trunc_a3m": os.path.join(base_dir, "Output/trunc_a3m"),
        "output_sorted_a3m": os.path.join(base_dir, "Output/sorted_a3m"),
        "output_combined_a3m": os.path.join(base_dir, "Output/combined_a3m"),
        "output_search": os.path.join(base_dir, "Output/PSSM")
    }
    
    for name, path in subdirs.items():
        os.makedirs(path, exist_ok=True)
        print(f"Directory '{path}' created.")
    
    # Define PAM matrix path
    pam_matrix_path = os.path.join(os.getcwd(), "PAM30.txt")
    input_sequences = os.path.join(base_dir, "Input/input.fasta")
    pssm_output_path_pam = os.path.join(base_dir, "pssm_PAM30.csv")
    pssm_output_path_blosum = os.path.join(base_dir, "pssm_BLOSUM62.csv")
    output_search_file = os.path.join(base_dir, "Output/PSSM/PSSM.fasta")
    output_search_file_nonred = os.path.join(base_dir, "Output/PSSM/PSSM_nonred.fasta")
    output_prey_bait = os.path.join(base_dir, "Output/PSSM/PreyBait.fasta")
    output_fasta_split_directory = os.path.join(base_dir, "Output/Fasta/")
    jackhmmer_output = os.path.join(base_dir, "Output/a3m")
    a3m_output = os.path.join(base_dir, "Output/a3m")
    sto_output = os.path.join(base_dir, "Output/a3m/sto")
    truncated_a3m_output = os.path.join(base_dir, "Output/trunc_a3m")
    sorted_dir = os.path.join(base_dir, "Output/sorted_a3m")
    combined_dir = os.path.join(base_dir, "Output/combined_a3m")

    # Print the configuration
    print("\nConfiguration:")
    print(f"Base Directory: {base_dir}")
    for key, value in subdirs.items():
        print(f"{key}: {value}")
    print(f"pam_matrix_path: {pam_matrix_path}")
    
    # Return the configuration
    return base_dir, subdirs, pam_matrix_path, input_sequences, pssm_output_path_pam, pssm_output_path_blosum, output_search_file, output_prey_bait, output_fasta_split_directory, jackhmmer_output, a3m_output, sto_output, truncated_a3m_output, sorted_dir, combined_dir, output_search_file_nonred

if __name__ == "__main__":
    # Set up the environment
    base_dir, subdirectories, pam_matrix_path, input_sequences, pssm_output_path_pam, pssm_output_path_blosum, output_search_file, output_prey_bait, output_fasta_split_directory, jackhmmer_output, a3m_output, sto_output, truncated_a3m_output, sorted_dir, combined_dir, output_search_file_nonred = setup_environment()


Project folder '/media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run' created.
Directory '/media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Input' created.
Directory '/media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/Fasta' created.
Directory '/media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/a3m' created.
Directory '/media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/trunc_a3m' created.
Directory '/media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/sorted_a3m' created.
Directory '/media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/combined_a3m' created.
Directory '/media/pythagoras/895c7a01-1c19

##### Move your input.fasta to the Input Folder

#### 1A. Create an PSSM PAM30 Matrix of the input Sequences  

In [4]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from collections import defaultdict

# Define the input and output file paths
filtered_sequences_path = input_sequences
pssm_output_path = pssm_output_path_pam
pam_matrix_path = pam_matrix_path

# Standard set of amino acids
standard_amino_acids = "ACDEFGHIKLMNPQRSTVWY"

# Function to read sequences from a FASTA file
def read_fasta_sequences(fasta_file):
    """
    Reads protein sequences from a FASTA file and converts them to uppercase.
    """
    sequences = [str(record.seq).upper() for record in SeqIO.parse(fasta_file, "fasta")]
    return sequences

# Function to calculate the frequency matrix
def calculate_frequency_matrix(sequences):
    """
    Calculates the frequency of each standard amino acid at each position across all sequences.
    """
    if not sequences:
        raise ValueError("No sequences found in the input file.")
        
    sequence_length = len(sequences[0])
    for seq in sequences:
        if len(seq) != sequence_length:
            raise ValueError("All sequences must be of the same length.")
    
    amino_acids = sorted(set(standard_amino_acids))
    
    # Initialize the frequency matrix with zeros
    frequency_matrix = np.zeros((len(amino_acids), sequence_length))
    aa_to_index = {aa: idx for idx, aa in enumerate(amino_acids)}
    
    # Count the frequency of each amino acid at each position
    for seq in sequences:
        for i, aa in enumerate(seq):
            if aa in aa_to_index:
                frequency_matrix[aa_to_index[aa], i] += 1
    
    # Normalize the frequency matrix by the number of sequences
    frequency_matrix /= len(sequences)
    
    return frequency_matrix, amino_acids

# Function to load PAM matrix from file
def load_pam_matrix(file_path):
    """
    Loads the PAM matrix from a file and converts it to a nested dictionary.
    """
    pam = defaultdict(dict)
    with open(file_path, 'r') as f:
        lines = f.readlines()
        # Skip header lines starting with '#'
        lines = [line.strip() for line in lines if not line.startswith('#') and line.strip()]
        header = lines[0].split()
        for line in lines[1:]:
            parts = line.split()
            aa1 = parts[0]
            scores = parts[1:]
            for aa2, score in zip(header, scores):
                pam[aa1][aa2] = int(score)
                pam[aa2][aa1] = int(score)  # Assuming symmetry
    return pam

# Function to generate the PSSM matrix from the frequency matrix using PAM250 scores
def generate_pssm_matrix(frequency_matrix, amino_acids, pam_matrix):
    """
    Generates a Position-Specific Scoring Matrix (PSSM) by applying the PAM250 substitution scores
    to the frequency matrix.
    """
    pssm_matrix = np.zeros(frequency_matrix.shape)
    
    for i in range(frequency_matrix.shape[1]):  # Iterate over each position
        for j, aa in enumerate(amino_acids):    # Iterate over each amino acid
            score = 0
            for k, aa2 in enumerate(amino_acids):  # Iterate over substitution pairs
                substitution_score = pam_matrix[aa].get(aa2, 0)
                score += frequency_matrix[k, i] * substitution_score
            pssm_matrix[j, i] = score
    
    return pssm_matrix

# Read the PAM250 matrix from the file
pam_matrix = load_pam_matrix(pam_matrix_path)

# Read the filtered sequences from the FASTA file
sequences = read_fasta_sequences(filtered_sequences_path)

# Calculate the frequency matrix
frequency_matrix, amino_acids = calculate_frequency_matrix(sequences)

# Generate the PSSM matrix using PAM250
pssm_matrix = generate_pssm_matrix(frequency_matrix, amino_acids, pam_matrix)

# Convert PSSM matrix to DataFrame for easier handling and export
pssm_df = pd.DataFrame(pssm_matrix, index=list(amino_acids))

# Ensure the DataFrame includes all standard amino acids
for aa in standard_amino_acids:
    if aa not in pssm_df.index:
        pssm_df.loc[aa] = [0] * pssm_df.shape[1]

# Reorder the DataFrame to match the standard amino acids order
pssm_df = pssm_df.loc[list(standard_amino_acids)]

# Output the PSSM matrix to a CSV file
pssm_df.to_csv(pssm_output_path)

print(f"PSSM matrix saved to: {pssm_output_path}")

PSSM matrix saved to: /media/michael/Michi-Storage/Postdoc-IBS/Themis_Actin-Peptide-clustering/Test/pssm_PAM30.csv


#### 1B. Create an PSSM BLOSUM62 Matrix of the input Sequences  

In [5]:
import numpy as np
import pandas as pd
from Bio import SeqIO
import blosum as bl

# Define the input and output file paths
filtered_sequences_path = input_sequences
pssm_output_path = pssm_output_path_blosum

# Load the BLOSUM62 matrix
matrix = bl.BLOSUM(62)

# Standard set of amino acids
standard_amino_acids = "ACDEFGHIKLMNPQRSTVWY"

# Function to read sequences from a FASTA file
def read_fasta_sequences(fasta_file):
    sequences = [str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")]
    return sequences

# Function to calculate the frequency matrix
def calculate_frequency_matrix(sequences):
    sequence_length = len(sequences[0])
    amino_acids = sorted(set(standard_amino_acids))
    
    # Initialize the frequency matrix with zeros
    frequency_matrix = np.zeros((len(amino_acids), sequence_length))
    aa_to_index = {aa: idx for idx, aa in enumerate(amino_acids)}
    
    # Count the frequency of each amino acid at each position
    for seq in sequences:
        for i, aa in enumerate(seq):
            if aa in aa_to_index:
                frequency_matrix[aa_to_index[aa], i] += 1
    
    # Normalize the frequency matrix by the number of sequences
    frequency_matrix /= len(sequences)
    
    return frequency_matrix, amino_acids

# Function to generate the PSSM matrix from the frequency matrix using BLOSUM62 scores
def generate_pssm_matrix(frequency_matrix, amino_acids, blosum62):
    pssm_matrix = np.zeros(frequency_matrix.shape)
    
    for i in range(frequency_matrix.shape[1]):
        for j, aa in enumerate(amino_acids):
            score = 0
            for k, aa2 in enumerate(amino_acids):
                score += frequency_matrix[k, i] * blosum62[aa][aa2]
            pssm_matrix[j, i] = score
    
    return pssm_matrix

# Read the filtered sequences from the FASTA file
sequences = read_fasta_sequences(filtered_sequences_path)

# Calculate the frequency matrix
frequency_matrix, amino_acids = calculate_frequency_matrix(sequences)

# Generate the PSSM matrix
pssm_matrix = generate_pssm_matrix(frequency_matrix, amino_acids, matrix)

# Convert PSSM matrix to DataFrame for easier handling and export
pssm_df = pd.DataFrame(pssm_matrix, index=list(amino_acids))

# Ensure the DataFrame includes all standard amino acids
for aa in standard_amino_acids:
    if aa not in pssm_df.index:
        pssm_df.loc[aa] = [0] * pssm_df.shape[1]

# Reorder the DataFrame to match the standard amino acids order
pssm_df = pssm_df.loc[list(standard_amino_acids)]

# Output the PSSM matrix to a CSV file
pssm_df.to_csv(pssm_output_path)

print(f"PSSM matrix saved to: {pssm_output_path}")

PSSM matrix saved to: /media/michael/Michi-Storage/Postdoc-IBS/Themis_Actin-Peptide-clustering/Test/pssm_BLOSUM62.csv


#### 2. Run the PSSM Search  

In [2]:
"""
Here, we will define the secondary structure optimization strategy and set the cutoff values.
"""

optimization = input("Please specify the optimization strategy for the secondary structure.\nType 'helix', 'strand', 'coil' or 'unknown'.").strip().lower()
if optimization == "helix":
    coil_cutoff = 0.9
    helix_cutoff = 0.1
    strand_cutoff = 0.9
elif optimization == "strand":
    coil_cutoff = 0.9
    helix_cutoff = 0.9
    strand_cutoff = 0.1
elif optimization == "coil":
    coil_cutoff = 0.1
    helix_cutoff = 0.9
    strand_cutoff = 0.9
elif optimization == "unknown":
    coil_cutoff = 0
    helix_cutoff = 0
    strand_cutoff = 0
else:
    raise ValueError("Invalid optimization strategy. Please specify 'helix', 'strand', 'coil' or 'unknown'.")

#Set the Cutoffs
pssm_cutoff = 0
iupred_cutoff = 0.40 
anchor_cutoff = 0.40

#Choose the Matrix you want to use, by removing the "#"
pssm_matrix_path = pssm_output_path_pam
#pssm_matrix_path = pssm_output_path_blosum

In [3]:
import numpy as np
import pandas as pd
import subprocess
import tempfile
import os
from Bio import SeqIO
from multiprocessing import Pool, cpu_count

# Define paths
fasta_file = NCBI_protein_database
output_fasta_file = output_search_file
iupred_path = iupred_path
psipred_path = psipred_path
pssm_matrix_path = pssm_matrix_path

# Function to read the PSSM matrix from a CSV file
def read_pssm_matrix(pssm_matrix_path):
    pssm_matrix = pd.read_csv(pssm_matrix_path, index_col=0)
    return pssm_matrix

# Function to read sequences from a FASTA file
def read_fasta_sequences(fasta_file):
    fasta_sequences = list(SeqIO.parse(fasta_file, "fasta"))
    return fasta_sequences

# Function to cut sequence into fragments
def cut_sequence_into_fragments(sequence, fragment_size):
    sequence_length = len(sequence)
    fragments = []
    for i in range(sequence_length - fragment_size + 1):
        fragment = sequence[i:i + fragment_size]
        fragments.append((i, fragment))  # Include the start index
    return fragments

# Function to get IUPRED and ANCHOR scores
def get_iupred_anchor_scores(sequence, seq_name, iupred_path):
    temp_fasta = tempfile.NamedTemporaryFile(delete=False, mode='w', suffix='.fasta')
    temp_fasta.write(f">{seq_name}\n{sequence}\n")
    temp_fasta.close()
    temp_fasta_name = temp_fasta.name

    try:
        # Run IUPred3 for disorder and ANCHOR scores
        iupred_cmd = [
            "python3",
            os.path.join(iupred_path, "iupred3.py"),
            temp_fasta_name,
            "long",
            "--anchor"
        ]
        result = subprocess.run(iupred_cmd, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"Error running IUPred3 for {seq_name}: {result.stderr}")
            return None, None

        iupred_scores, anchor_scores = [], []
        for line in result.stdout.splitlines():
            if line.startswith('#') or line.strip() == '':
                continue
            parts = line.split()
            if len(parts) == 4:
                iupred_scores.append(float(parts[2]))  # IUPRED score
                anchor_scores.append(float(parts[3]))  # ANCHOR score
        return iupred_scores, anchor_scores
    finally:
        os.unlink(temp_fasta_name)

# Function to get secondary structure probabilities from PSIPRED
def get_psipred_probabilities(seq_id, sequence, psipred_path):
    with tempfile.TemporaryDirectory() as temp_dir:
        original_dir = os.getcwd()
        try:
            os.chdir(temp_dir)
            temp_fasta_name = 'input.fasta'
            with open(temp_fasta_name, 'w') as temp_fasta:
                temp_fasta.write(f">{seq_id}\n{sequence}\n")
            
            env = os.environ.copy()
            env['PSIPRED_DATA'] = os.path.join(psipred_path, 'data')
            env['PATH'] += ':/usr/bin'  # Ensure BLAST is available in PATH
            
            runpsipred_cmd = [
                os.path.join(psipred_path, "runpsipred_single"),
                temp_fasta_name
            ]
            result = subprocess.run(runpsipred_cmd, capture_output=True, text=True, env=env)
            
            if result.returncode != 0:
                print(f"Error running PSIPRED for {seq_id}: {result.stderr}")
                return None

            ss2_file = 'input.ss2'
            if not os.path.exists(ss2_file):
                return None

            probabilities = []
            with open(ss2_file, 'r') as f:
                for line in f:
                    if line.startswith('#') or line.strip() == '':
                        continue
                    parts = line.strip().split()
                    if len(parts) != 6:
                        continue
                    prob_c = float(parts[3])  # Coil
                    prob_h = float(parts[4])  # Helix
                    prob_e = float(parts[5])  # Strand
                    probabilities.append({'Prob_C': prob_c, 'Prob_H': prob_h, 'Prob_E': prob_e})
            return probabilities
        finally:
            os.chdir(original_dir)

def pssm_score_sequence(sequence, pssm_matrix):
    valid_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")  # The standard 20 amino acids
    score = 0
    for i in range(len(sequence)):
        aa = sequence[i]
        column_name = str(i)
        if aa not in valid_amino_acids:
            print(f"Skipping fragment due to invalid amino acid '{aa}' in sequence '{sequence}'")
            return None
        if aa in pssm_matrix.index and column_name in pssm_matrix.columns:
            score += pssm_matrix.at[aa, column_name]
        else:
            print(f"Invalid amino acid '{aa}' or column '{column_name}' in sequence '{sequence}'")
            return None
    return score

# Function to process a single sequence record, modified to collect all hits
def process_sequence_record(args):
    (seq_record, pssm_matrix, iupred_path, psipred_path, pssm_cutoff, 
     iupred_cutoff, anchor_cutoff, ss_cutoff, flanking_aa_size, flank_size, optimization) = args
    sequence_name = seq_record.id
    full_header = seq_record.description
    sequence = str(seq_record.seq)

    # Initialize variables to store IUPRED and ANCHOR scores, and PSIPRED probabilities
    iupred_scores = None
    anchor_scores = None
    psipred_probs = None

    fragment_size = pssm_matrix.shape[1]
    fragments = cut_sequence_into_fragments(sequence, fragment_size)

    hits = []  # List to store all hits for the current sequence

    # Loop over fragments
    for i, fragment in fragments:
        pssm_score = pssm_score_sequence(fragment, pssm_matrix)

        # Check if the pssm_score is valid before proceeding
        if pssm_score is None:
            continue  # Skip this fragment if pssm_score is None

        if pssm_score >= pssm_cutoff:
            # If IUPRED and ANCHOR scores are not computed yet, compute them for the whole sequence
            if iupred_scores is None or anchor_scores is None:
                iupred_scores, anchor_scores = get_iupred_anchor_scores(sequence, sequence_name, iupred_path)
                if iupred_scores is None or anchor_scores is None:
                    print(f"Skipping sequence {sequence_name} due to IUPRED/ANCHOR failure.")
                    return None  # Skip this sequence if IUPRED/ANCHOR failed

            # Define flanking regions
            start_flank = max(0, i - flank_size)
            end_flank = min(len(sequence), i + fragment_size + flank_size)

            # Exclude the fragment itself from the IUPRED score calculation
            left_flank_scores = iupred_scores[start_flank:i]  # Flanking region before the fragment
            right_flank_scores = iupred_scores[i + fragment_size:end_flank]  # Flanking region after the fragment
            flanking_region_scores = left_flank_scores + right_flank_scores

            if len(flanking_region_scores) == 0:
                mean_iupred = 0  # Adjust as needed
            else:
                mean_iupred = np.mean(flanking_region_scores)

            # Calculate mean ANCHOR score over the fragment itself
            fragment_anchor_scores = anchor_scores[i:i+fragment_size]
            mean_anchor = np.mean(fragment_anchor_scores)

            if mean_iupred >= iupred_cutoff and mean_anchor >= anchor_cutoff:
                # If PSIPRED probabilities are not computed yet, compute them for the whole sequence
                if psipred_probs is None:
                    psipred_probs = get_psipred_probabilities(sequence_name, sequence, psipred_path)
                    if psipred_probs is None:
                        print(f"Skipping sequence {sequence_name} due to PSIPRED failure.")
                        return None  # Skip this sequence if PSIPRED failed

                # Extract probabilities for the fragment positions
                fragment_psipred_probs = psipred_probs[i:i+fragment_size]

                mean_coil = np.mean([p['Prob_C'] for p in fragment_psipred_probs])
                mean_helix = np.mean([p['Prob_H'] for p in fragment_psipred_probs])
                mean_strand = np.mean([p['Prob_E'] for p in fragment_psipred_probs])

                fragment_passes_filters = False  # Initialize to False
                if optimization == "helix":
                    if (mean_coil < ss_cutoff['coil'] and mean_helix >= ss_cutoff['helix'] and mean_strand < ss_cutoff['strand']):
                        fragment_passes_filters = True

                elif optimization == "strand":
                    if (mean_coil < ss_cutoff['coil'] and mean_helix < ss_cutoff['helix'] and mean_strand >= ss_cutoff['strand']):
                        fragment_passes_filters = True


                elif optimization == "coil":
                    if (mean_coil >= ss_cutoff['coil'] and mean_helix < ss_cutoff['helix'] and mean_strand < ss_cutoff['strand']):
                        fragment_passes_filters = True
 
                elif optimization == "unknown":
                    if (mean_coil > ss_cutoff['coil'] and mean_helix > ss_cutoff['helix'] and mean_strand > ss_cutoff['strand']):    
                        fragment_passes_filters = True # 

                else:
                    raise ValueError("Invalid optimization strategy. Please specify 'helix', 'strand', 'coil' or 'unknown'.")
                
                if fragment_passes_filters:
                    # Include flanking amino acids from the original sequence
                    start_output_flank = max(0, i - flanking_aa_size)
                    end_output_flank = min(len(sequence), i + fragment_size + flanking_aa_size)
                    fragment_with_output_flanks = sequence[start_output_flank:end_output_flank]

                    fasta_header = (f">{full_header} pos:{i+1} PSSM_Score:{pssm_score:.2f} "
                                    f"IUPRED_Score:{mean_iupred:.2f} ANCHOR_Score:{mean_anchor:.2f} "
                                    f"Coil:{mean_coil:.2f} Helix:{mean_helix:.2f} Strand:{mean_strand:.2f}")

                    # Append this fragment to the hits list
                    hits.append((fasta_header, fragment_with_output_flanks))

    return hits  # Return the list of hits for this sequence

def main(pssm_cutoff, iupred_cutoff, anchor_cutoff, coil_cutoff, helix_cutoff, strand_cutoff):
    # Load PSSM matrix
    pssm_matrix = read_pssm_matrix(pssm_matrix_path)
    print(f"PSSM Matrix Columns: {pssm_matrix.columns.tolist()}")

    # Read the sequences from the FASTA file
    sequences_to_score = read_fasta_sequences(fasta_file)

    # Define the cutoffs and parameters
    pssm_cutoff, iupred_cutoff, anchor_cutoff = pssm_cutoff, iupred_cutoff, anchor_cutoff 
    ss_cutoff = {'coil': coil_cutoff, 'helix': helix_cutoff, 'strand': strand_cutoff}
    flanking_aa_size = 20  # Number of flanking amino acids to include when writing output
    flank_size = 60       # Flank size for IUPRED calculation (same as in Code B)

    # Create args list
    args = [
        (
            seq_record, pssm_matrix, iupred_path, psipred_path, 
            pssm_cutoff, iupred_cutoff, anchor_cutoff, ss_cutoff, 
            flanking_aa_size, flank_size, optimization
        ) 
        for seq_record in sequences_to_score
    ]

    # Perform parallel processing
    num_cores = cpu_count() - 1
    print(f"Using {num_cores} cores for processing.")
    with Pool(num_cores) as pool:
        all_results = pool.map(process_sequence_record, args)

    # Write all hits from each sequence to the output file
    with open(output_fasta_file, "w") as output_conn:
        for result in all_results:
            if result:  # Only write if there were hits found
                for header, fragment in result:
                    output_conn.write(f"{header}\n{fragment}\n")

    print(f"All hits written to {output_fasta_file}")

if __name__ == "__main__":
    main(pssm_cutoff, iupred_cutoff, anchor_cutoff, coil_cutoff, helix_cutoff, strand_cutoff)

PSSM Matrix Columns: ['0', '1', '2', '3', '4', '5', '6', '7', '8']
Using 47 cores for processing.
Skipping fragment due to invalid amino acid 'U' in sequence 'VVALLQASU'
Skipping fragment due to invalid amino acid 'U' in sequence 'VALLQASUY'
Skipping fragment due to invalid amino acid 'U' in sequence 'ALLQASUYL'
Skipping fragment due to invalid amino acid 'U' in sequence 'LLQASUYLC'
Skipping fragment due to invalid amino acid 'U' in sequence 'LQASUYLCI'
Skipping fragment due to invalid amino acid 'U' in sequence 'QASUYLCIL'
Skipping fragment due to invalid amino acid 'U' in sequence 'ASUYLCILQ'
Skipping fragment due to invalid amino acid 'U' in sequence 'SUYLCILQA'
Skipping fragment due to invalid amino acid 'U' in sequence 'UYLCILQAS'
Skipping fragment due to invalid amino acid 'U' in sequence 'DSELAPRSU'
Skipping fragment due to invalid amino acid 'U' in sequence 'SELAPRSUC'
Skipping fragment due to invalid amino acid 'U' in sequence 'ELAPRSUCC'
Skipping fragment due to invalid amino

In [6]:
"""
Remove redundant sequences in the PSSM-hits file of the i-th iteration.
This avoids running identical sequences through jackhmmer and ColabFold multiple times.
"""


def read_fasta_sequences(file_path):
    """
    Reads sequences from a FASTA file and returns a dictionary with sequences as keys and headers as values.
    If a sequence appears multiple times with different headers, only the first header is kept.
    """
    sequences = {}
    with open(file_path, "r") as file:
        header = None
        sequence_data = []
        for line in file:
            if line.startswith(">"):
                # If there is an existing sequence, add it to the dictionary
                if header and sequence_data:
                    sequence = "".join(sequence_data)
                    # Add the sequence to the dictionary if it's not already present
                    if sequence not in sequences:
                        sequences[sequence] = header
                # Start a new sequence
                header = line.strip()  # Keep the full header, including '>'
                sequence_data = []
            else:
                sequence_data.append(line.strip())
        # Add the last sequence to the dictionary
        if header and sequence_data:
            sequence = "".join(sequence_data)
            if sequence not in sequences:
                sequences[sequence] = header
    return sequences


if __name__ == "__main__":
    # Define the input and output file paths
    input_file_path = output_search_file  
    output_file_path = output_search_file_nonred

    # Read the sequences from the input file
    sequences_dict = read_fasta_sequences(input_file_path)

    # Write the unique sequences with their original headers to the output file
    with open(output_file_path, "w") as output_file:
        for sequence, header in sequences_dict.items():
            output_file.write(f"{header}\n{sequence}\n")

    print(f"{len(sequences_dict)} non-redundant sequences written to: {output_file_path}")


1976 non-redundant sequences written to: /media/michael/Michi-Storage/Postdoc-IBS/Themis_Actin-Peptide-clustering/Test/Output/PSSM/PSSM_nonred.fasta


#### (Optional)If this is not the first iteration, it would be wise to remove already found and analysed sequences to reduce computional time and costs. If this is the first iteration just skip the following code

In [ ]:
"""
This will compare the PSSM-hits of two iterations and write the unique hits to a new FASTA file.
Please ignore this cell in case you are running the first iteration.
"""


# File paths for your two FASTA files
file_iteration01 = "/home/pythagoras/Documents/PhD/PSSMSearch/All_Iterations_nonred.fasta"
file_iteration02 = "/home/pythagoras/Documents/PhD/PSSMSearch/Iteration4/Output/PSSM/PSSM.fasta"
output_file = "/home/pythagoras/Documents/PhD/PSSMSearch/Iteration4/Output/PSSM/Unique_PSSM.fasta"  # Output file for unique sequences

def read_fasta_sequences(file_path):
    """
    Reads sequences from a FASTA file and returns a dictionary with sequence IDs as keys and sequences as values.
    """
    sequences = {}
    with open(file_path, "r") as file:
        sequence_id = None
        sequence_data = []
        for line in file:
            if line.startswith(">"):
                if sequence_id and sequence_data:
                    sequences[sequence_id] = "".join(sequence_data)
                sequence_id = line[1:].strip()
                sequence_data = []
            else:
                sequence_data.append(line.strip())
        if sequence_id and sequence_data:
            sequences[sequence_id] = "".join(sequence_data)
    return sequences


# Read sequences from both files
sequences_iteration01 = read_fasta_sequences(file_iteration01)
sequences_iteration02 = read_fasta_sequences(file_iteration02)

# Create a set of sequences in Iteration 01 to check against
sequences_set_iteration01 = set(sequences_iteration01.values())

# Find new hits in Iteration 02 that have unique sequences (not in Iteration 01)
unique_sequences_iteration02 = {
    seq_id: seq
    for seq_id, seq in sequences_iteration02.items()
    if seq not in sequences_set_iteration01
}

# Write unique sequences to a new FASTA file
with open(output_file, "w") as output:
    for seq_id, sequence in unique_sequences_iteration02.items():
        output.write(f">{seq_id}\n{sequence}\n")

print(f"Unique sequences have been written to {output_file}")

Unique sequences have been written to /home/pythagoras/Documents/PhD/PSSMSearch/Iteration4/Output/PSSM/Unique_PSSM.fasta


#### 3. Adds the bait protein sequence to each FASTA-entry in the form "Protein:Bait-seq"

In [2]:
#Add the path of the cleared non redundant fasta file from the PSSM Search
input_fasta_path = output_search_file_nonred

def process_fasta(input_path, output_path, bait_sequence, max_length=None):
    # Read and format the sequences
    formatted_sequences = []
    
    with open(input_path, "r") as file:
        protein_name = None
        protein_sequence = []
        
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if protein_name:  # Save the previous protein sequence if exists
                    combined_sequence = "".join(protein_sequence) + ":" + bait_sequence
                    formatted_sequences.append((protein_name, combined_sequence))
                    
                protein_name = line  # Start a new protein sequence
                protein_sequence = []
            else:
                protein_sequence.append(line)
    
        # Handle the last protein sequence
        if protein_name:
            combined_sequence = "".join(protein_sequence) + ":" + bait_sequence
            formatted_sequences.append((protein_name, combined_sequence))

    # Optionally curate the sequences based on length
    if max_length:
        formatted_sequences = [(name, seq) for name, seq in formatted_sequences if len(seq.split(":")[0]) <= max_length]

    # Write the formatted (and possibly curated) sequences to the output file
    with open(output_path, "w") as outfile:
        for name, seq in formatted_sequences:
            outfile.write(name + "\n" + seq + "\n")

    return formatted_sequences

# Usage example
bait_sequence = bait_sequence
output_fasta_path = output_prey_bait
max_len = 3000  # Optional: set to None if you don't want to curate based on length

processed_sequences = process_fasta(input_fasta_path, output_fasta_path, bait_sequence, max_len)


#### 4. Splits a combined FASTA file into individual files for each sequence, naming the files based on the accession number and position extracted from the FASTA headers.

In [3]:
"""
This will split the PreyBait.fasta file into individual FASTA files to generate the input for ColabFold.
"""

import re
import os


def sanitize_filename(filename):
    """Sanitizes the filename by replacing any characters that are not allowed in filenames."""
    s = re.sub(r'[^0-9a-zA-Z_\.]+', '_', filename)
    return s.strip('_')


def extract_accession_and_position(identifier):
    """
    Extracts the accession number and position from the FASTA header.
    """
    # Remove the leading '>' if present
    identifier = identifier.lstrip('>')

    # Use regex to find the accession number and position
    match = re.search(r'^(\S+).*?pos:(\d+)', identifier)
    if match:
        accession = match.group(1)  # Extract accession number (e.g., NP_001375333.1)
        position = match.group(2)   # Extract position number (e.g., 143)
        filename = f"{accession}_pos_{position}"
        return sanitize_filename(filename)
    else:
        # Fallback: sanitize and shorten the entire identifier
        return sanitize_filename(identifier)[:50]  # Limit to 50 characters to avoid long filenames


def split_fasta_to_files(input_file, output_directory):
    """
    Splits a combined FASTA file into individual files for each sequence, naming the files based on accession number and position.

    Parameters:
    - input_file: Path to the combined FASTA file.
    - output_directory: Directory where individual FASTA files will be saved.
    - max_entries (optional): Maximum number of FASTA entries to process. Defaults to processing all entries.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Read the combined FASTA file
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # Process the lines to split into identifiers and sequences
    identifiers = [lines[i].strip() for i in range(0, len(lines), 2)]
    sequences = [lines[i].strip() for i in range(1, len(lines), 2)]

    # Extract filenames based on accession number and position
    filenames = [extract_accession_and_position(identifier) for identifier in identifiers]

    # Write individual FASTA files
    for filename, identifier, sequence in zip(filenames, identifiers, sequences):
        filepath = os.path.join(output_directory, f"{filename}.fasta")

        with open(filepath, 'w') as file:
            file.write(f"{identifier}\n{sequence}\n")


# Example Usage
input_path = output_prey_bait
output_directory = output_fasta_split_directory

split_fasta_to_files(input_path, output_directory)

#### 5. Runs Jackhmmer for Bait

In [4]:
import re
import os
import subprocess
import logging
from pathlib import Path
from multiprocessing import cpu_count


# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')

def sanitize_filename(filename):
    """Sanitizes the filename by replacing any characters that are not allowed in filenames."""
    s = re.sub(r'[^0-9a-zA-Z_\.]+', '_', filename)
    return s.strip('_')

def safe_makedirs(directory):
    """Creates a directory safely, avoiding race conditions."""
    Path(directory).mkdir(parents=True, exist_ok=True)

if __name__ == "__main__":
    # Ensure the necessary variables are defined
    try:
        bait_sequence
        uniref90_path
        jackhmmer_output
    except NameError as e:
        logging.error(f"Required variable not defined: {e}")
        raise

    # Prepare the bait sequence
    bait_sequence_header = ">bait_sequence"
    sequence = bait_sequence.strip()

    # Sanitize header to create filename
    filename = sanitize_filename(bait_sequence_header.lstrip('>'))

    # Define output directories
    output_dir = jackhmmer_output  # Should be defined from setup_environment()
    sto_dir = os.path.join(output_dir, "sto")
    tbl_dir = os.path.join(output_dir, "tbl")
    safe_makedirs(sto_dir)
    safe_makedirs(tbl_dir)

    sto_output_file = os.path.join(sto_dir, f"{filename}.sto")
    tbl_output_file = os.path.join(tbl_dir, f"{filename}_output.tbl")

    fasta_file = f"{filename}.fasta"
    with open(fasta_file, 'w') as f:
        f.write(f"{bait_sequence_header}\n{sequence}\n")

    # Run jackhmmer
    # Number of CPUs to use
    num_cpus = cpu_count() - 1  # Adjust as needed

    # Use the -E option to restrict based on E-value
    cmd = (
    f"jackhmmer --cpu {num_cpus} -E 0.0001 -N 1 "
    f"--F1 0.0005 --F2 0.00005 --F3 0.0000005 --noali "
    f"--tblout {tbl_output_file} -A {sto_output_file} {fasta_file} {uniref90_path}")


    try:
        logging.info(f"Running jackhmmer for bait sequence")
        subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        logging.info(f"Completed jackhmmer for bait sequence")
    except subprocess.CalledProcessError as e:
        logging.error(f"Jackhmmer failed for bait sequence: {e}")
    finally:
        os.remove(fasta_file)


2024-12-13 09:42:12,205 [INFO] Running jackhmmer for bait sequence
2024-12-13 09:46:31,844 [INFO] Completed jackhmmer for bait sequence


#### 6. Runs Jackhmmer for Peptides

In [7]:
import re
import os
import subprocess
import logging
from multiprocessing import Pool, cpu_count, Manager, Lock
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')

def sanitize_filename(filename):
    """Sanitizes the filename by replacing any characters that are not allowed in filenames."""
    s = re.sub(r'[^0-9a-zA-Z_\.]+', '_', filename)
    return s.strip('_')

def extract_accession_and_position(identifier):
    """
    Extracts the accession number and position from the FASTA header.
    """
    identifier = identifier.lstrip('>')
    match = re.search(r'^(\S+).*?pos:(\d+)', identifier)
    if match:
        accession = match.group(1)
        position = match.group(2)
        filename = f"{accession}_pos_{position}"
        return sanitize_filename(filename)
    else:
        return sanitize_filename(identifier)[:50]

def safe_makedirs(directory):
    """Creates a directory safely, avoiding race conditions."""
    Path(directory).mkdir(parents=True, exist_ok=True)

def write_remaining_fasta(remaining_dict, remaining_fasta_path):
    """
    Writes the current sequences in remaining_dict to the input_remaining.fasta file.
    remaining_dict is a dictionary: {header: sequence}
    """
    with open(remaining_fasta_path, 'w') as f:
        for header, seq in remaining_dict.items():
            f.write(f"{header}\n{seq}\n")

def run_jackhmmer(args):
    """
    Runs jackhmmer on the given sequence and writes the output to a file.
    Updates the input_remaining.fasta upon successful completion.
    """
    (sequence, header, output_dir, seqdb, num_cpus, lock, remaining_fasta_path, remaining_dict) = args

    sto_dir = os.path.join(output_dir, "sto")
    tbl_dir = os.path.join(output_dir, "tbl")
    safe_makedirs(sto_dir)
    safe_makedirs(tbl_dir)

    filename = extract_accession_and_position(header)
    sto_output_file = os.path.join(sto_dir, f"{filename}.sto")
    tbl_output_file = os.path.join(tbl_dir, f"{filename}_output.tbl")

    # Temporary FASTA file for this sequence
    fasta_file = f"{filename}.fasta"
    with open(fasta_file, 'w') as f:
        f.write(f"{header}\n{sequence}\n")

    cmd = (
    f"jackhmmer --cpu {num_cpus} -E 0.0001 -N 1 "
    f"--F1 0.0005 --F2 0.00005 --F3 0.0000005 --noali "
    f"-A {sto_output_file} --tblout {tbl_output_file} {fasta_file} {seqdb}")


    try:
        logging.info(f"Running jackhmmer for {header}")
        subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        logging.info(f"Completed jackhmmer for {header}")

        # On success, remove this sequence from the remaining_dict and update the file
        with lock:
            if header in remaining_dict:
                del remaining_dict[header]
                write_remaining_fasta(remaining_dict, remaining_fasta_path)

    except subprocess.CalledProcessError as e:
        logging.error(f"Jackhmmer failed for {header}: {e}")
    finally:
        os.remove(fasta_file)

    return sto_output_file, tbl_output_file

def parse_fasta(fasta_file):
    """
    Parses the FASTA file and returns a list of (sequence, header) tuples.
    """
    sequences = []
    with open(fasta_file, 'r') as f:
        header = None
        sequence = ""
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if header and sequence:
                    sequences.append((sequence, header))
                header = line
                sequence = ""
            else:
                sequence += line
        # Add the last sequence
        if header and sequence:
            sequences.append((sequence, header))
    return sequences

if __name__ == "__main__":
    # Define the input FASTA file, output directory, and the sequence database
    fasta_file = output_search_file_nonred
    output_dir = jackhmmer_output
    seqdb = uniref90_path  # Path to the database you want to search against

    # Create the output directory if it doesn't exist
    safe_makedirs(output_dir)

    # Parse the FASTA file to get the sequences
    sequences = parse_fasta(fasta_file)

    # Initialize the remaining sequences dictionary {header: sequence}
    # This will help us keep track of what still needs to be run.
    manager = Manager()
    remaining_dict = manager.dict({header: seq for (seq, header) in sequences})

    # Write initial remaining.fasta file containing all sequences
    remaining_fasta_path = os.path.join(os.path.dirname(fasta_file), "input_remaining.fasta")
    with open(remaining_fasta_path, 'w') as f:
        for seq, header in sequences:
            f.write(f"{header}\n{seq}\n")

    # Number of CPUs to use
    total_cpus = cpu_count()
    # Number of CPUs per jackhmmer process
    num_cpus_per_process = 1         # Adjust as needed
    # Number of jackhmmer processes to run in parallel
    num_processes = (total_cpus // num_cpus_per_process) - 1

    lock = manager.Lock()

    # Prepare arguments for run_jackhmmer
    args_list = [(seq, header, output_dir, seqdb, num_cpus_per_process, lock, remaining_fasta_path, remaining_dict) 
                 for seq, header in sequences]

    # Preload the database using vmtouch
    # This command will "touch" all pages of seqdb to bring them into RAM if possible.
    #vmtouch_cmd = ["vmtouch", "-t", seqdb]
    #logging.info("Preloading database into RAM using vmtouch...")
    #subprocess.run(vmtouch_cmd, check=True)
    #logging.info("Finished preloading database into RAM.")

    # Run jackhmmer in parallel
    with Pool(processes=num_processes) as pool:
        pool.map(run_jackhmmer, args_list)


2024-12-13 09:49:59,346 [INFO] Running jackhmmer for >NP_065821.1 cingulin [Homo sapiens] pos:178 PSSM_Score:13.25 IUPRED_Score:0.76 ANCHOR_Score:0.91 Coil:0.10 Helix:0.90 Strand:0.00
2024-12-13 09:49:59,346 [INFO] Running jackhmmer for >NP_065910.3 protein Shroom3 [Homo sapiens] pos:781 PSSM_Score:5.50 IUPRED_Score:0.82 ANCHOR_Score:0.87 Coil:0.05 Helix:0.95 Strand:0.01
2024-12-13 09:49:59,346 [INFO] Running jackhmmer for >NP_004454.2 FYVE, RhoGEF and PH domain-containing protein 1 [Homo sapiens] pos:211 PSSM_Score:24.75 IUPRED_Score:0.85 ANCHOR_Score:0.98 Coil:0.08 Helix:0.92 Strand:0.00
2024-12-13 09:49:59,346 [INFO] Running jackhmmer for >NP_002211.1 inositol-trisphosphate 3-kinase A [Homo sapiens] pos:31 PSSM_Score:10.75 IUPRED_Score:0.80 ANCHOR_Score:0.88 Coil:0.02 Helix:0.97 Strand:0.00
2024-12-13 09:49:59,347 [INFO] Running jackhmmer for >NP_001128668.1 cardiac-enriched FHL2-interacting protein [Homo sapiens] pos:143 PSSM_Score:14.62 IUPRED_Score:0.61 ANCHOR_Score:0.75 Coil:0.1

#### 7. Convert the .sto to .a3m

In [14]:
import os
import subprocess
import multiprocessing

def convert_sto_to_a3m(sto_file):
    """
    Converts a .sto file to an .a3m file using reformat.pl.
    Parameters:
    - sto_file: Path to the input .sto file.
    """
    # Define the script path and output directory inside the function
    script_path = reformat_path
    output_dir = a3m_output
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Extract the filename without the extension
    filename = os.path.basename(sto_file).replace(".sto", "")
    # Define the output .a3m file path
    a3m_file = os.path.join(output_dir, f"{filename}.a3m")
    # Build the reformat.pl command
    cmd = f"perl {script_path}/reformat.pl sto a3m {sto_file} {a3m_file}"
    # Run the command
    subprocess.run(cmd, shell=True)
    print(f"Converted {sto_file} to {a3m_file}")
    return a3m_file

def process_sto_files(sto_dir, num_processes):
    """
    Processes all .sto files in a directory and converts them to .a3m files in parallel.
    Parameters:
    - sto_dir: Directory containing the .sto files.
    - num_processes: Number of worker processes to use.
    """
    # Get a list of all .sto files
    sto_files = [os.path.join(sto_dir, filename) for filename in os.listdir(sto_dir) if filename.endswith(".sto")]
    # Use multiprocessing Pool to process files in parallel
    with multiprocessing.Pool(processes=num_processes) as pool:
        pool.map(convert_sto_to_a3m, sto_files)

# Define the directory containing .sto files
sto_dir = sto_output
# Specify the number of cores you want to use
num_cores = 12  # Change this to the desired number of cores
# Process all .sto files in the directory
process_sto_files(sto_dir, num_cores)

Converted /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/a3m/sto/XP001_pos_001.sto to /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/a3m/XP001_pos_001.a3mConverted /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/a3m/sto/XP003_pos_001.sto to /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/a3m/XP003_pos_001.a3m
Converted /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/a3m/sto/NP_004454.2_pos_211.sto to /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/a3m/NP_004454.2_pos_211.a3mConverted /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/a3m/sto/NP_001295299.1_pos_

#### 8. Sorts the .a3m file after sequence identity

In [18]:
import os
import shutil
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed

# Define paths
input_dir = a3m_output
sorted_dir = sorted_dir

# Ensure output directories exist
os.makedirs(sorted_dir, exist_ok=True)

def calculate_sequence_identity(seq1, seq2):
    """Calculates the global sequence identity between two sequences, comparing up to the shortest sequence length."""
    # Determine the minimum length to use for comparison
    min_length = min(len(seq1), len(seq2))
    
    # Convert sequences to uppercase for case-insensitive comparison
    seq1_upper = seq1[:min_length].upper()
    seq2_upper = seq2[:min_length].upper()

    # Count matches over the aligned portion, including gaps as mismatches
    matches = sum(res1 == res2 for res1, res2 in zip(seq1_upper, seq2_upper))
    identity = matches / min_length  # Calculate identity based on the aligned length
    return identity

def parse_a3m_to_dataframe(file_path):
    """Parses an .a3m file and warns if sequences differ in length from the reference."""
    headers = []
    sequences = []
    reference_length = None  # Will be set based on the first sequence
    
    with open(file_path, "r") as file:
        header = ''
        seq_lines = []
        
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if header and seq_lines:
                    # Join sequence lines and remove extra whitespace
                    seq = ''.join(seq_lines).replace(" ", "").replace("\t", "")
                    headers.append(header)
                    sequences.append(seq)
                    
                    # Set the reference length based on the first sequence
                    if reference_length is None:
                        reference_length = len(seq)
                        
                header = line[1:].split()[0]
                seq_lines = []
            else:
                seq_lines.append(line)
        
        # Add the last sequence
        if header and seq_lines:
            seq = ''.join(seq_lines).replace(" ", "").replace("\t", "")
            headers.append(header)
            sequences.append(seq)
            if reference_length is None:
                reference_length = len(seq)
    
    # Convert to DataFrame without padding or trimming
    df = pd.DataFrame({'header': headers, 'sequence': sequences})
    return df

def sort_a3m_file(args):
    """Reads an .a3m file, sorts sequences by global sequence identity to the reference sequence, and saves the sorted file."""
    file_path, output_dir = args
    print(f"Processing file: {file_path}")
    df = parse_a3m_to_dataframe(file_path)

    if df.empty:
        print(f"No sequences found in {file_path}. Skipping.")
        return

    reference_seq = df.iloc[0]['sequence']
    reference_header = df.iloc[0]['header']
    print(f"Reference sequence length: {len(reference_seq)}")

    # Calculate sequence identity scores relative to the reference sequence
    try:
        df['similarity'] = df['sequence'].apply(
            lambda x: calculate_sequence_identity(reference_seq, x))
        # Explicitly set the similarity of the reference sequence to a maximum value
        df.loc[df['header'] == reference_header, 'similarity'] = np.inf
    except Exception as e:
        print(f"Error calculating similarity for {file_path}: {e}")
        return

    # Sort sequences by similarity in descending order
    df_sorted = df.sort_values('similarity', ascending=False).reset_index(drop=True)
    print(f"Number of sequences after sorting: {len(df_sorted)}")

    # Save sorted sequences to the output directory with the original filename
    sorted_file_path = os.path.join(output_dir, os.path.basename(file_path))
    print(f"Attempting to save sorted file to: {sorted_file_path}")
    try:
        with open(sorted_file_path, "w") as file:
            for _, row in df_sorted.iterrows():
                file.write(f">{row['header']}\n{row['sequence']}\n")
        print(f"Sorted file saved as: {sorted_file_path}")
    except Exception as e:
        print(f"An error occurred while writing the file: {sorted_file_path}")
        print(f"Error message: {e}")
        import traceback
        traceback.print_exc()

def main():
    # Ask the user whether to sort 'bait_sequence.a3m'
    sort_bait_sequence_input = input("Warning: Sorting your a3m files might dilute the coevolutionary signals and impair the detection of interacting residues. Make sure to have a sufficient sequence variability after sorting to prevent loss of protein complex prediction accuracy.\nDo you want to sort 'bait_sequence.a3m'? (y/n): ").strip().lower()
    sort_bait_sequence = sort_bait_sequence_input in ['y', 'yes']

    # Prepare a list of files to process and their corresponding output directories
    tasks = []
    for filename in os.listdir(input_dir):
        if filename.endswith(".a3m"):
            file_path = os.path.join(input_dir, filename)
            # Determine the output directory based on the filename
            output_dir = sorted_dir
            if filename == "bait_sequence.a3m":
                if sort_bait_sequence:
                    print(f"Scheduling 'bait_sequence.a3m' for processing.")
                    tasks.append((file_path, output_dir))
                else:
                    # Skip processing and copy the file directly
                    destination_path = os.path.join(output_dir, filename)
                    shutil.copy(file_path, destination_path)
                    print(f"Copied 'bait_sequence.a3m' directly to {destination_path}")
            else:
                print(f"Scheduling other .a3m file for processing: {filename}")
                tasks.append((file_path, output_dir))

    # Set up the process pool
    num_workers = os.cpu_count() or 1  # Use the number of available CPU cores
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Submit tasks to the executor
        future_to_file = {executor.submit(sort_a3m_file, task): task for task in tasks}
        # As each task completes, you can handle results or exceptions
        for future in as_completed(future_to_file):
            task = future_to_file[future]
            try:
                future.result()
            except Exception as e:
                print(f"An error occurred while processing {task[0]}: {e}")

if __name__ == "__main__":
    main()


Scheduling other .a3m file for processing: NP_001033043.1_pos_198.a3m
Scheduling other .a3m file for processing: XP001_pos_001.a3m
Scheduling other .a3m file for processing: NP_002211.1_pos_31.a3m
Scheduling other .a3m file for processing: NP_001277260.1_pos_748.a3m
Scheduling other .a3m file for processing: NP_001239264.1_pos_353.a3m
Scheduling other .a3m file for processing: XP002_pos_001.a3m
Scheduling other .a3m file for processing: NP_065821.1_pos_178.a3m
Scheduling other .a3m file for processing: NP_001291410.1_pos_105.a3m
Scheduling other .a3m file for processing: NP_001307366.1_pos_1072.a3m
Scheduling other .a3m file for processing: NP_001159632.1_pos_202.a3m
Scheduling other .a3m file for processing: NP_065910.3_pos_781.a3m
Scheduling other .a3m file for processing: NP_001364348.1_pos_595.a3m
Scheduling 'bait_sequence.a3m' for processing.
Scheduling other .a3m file for processing: NP_001375333.1_pos_143.a3m
Scheduling other .a3m file for processing: NP_001128668.1_pos_143.a3m


#### 9. Trims the Number of Sequences

In [19]:
import os

# Define global parameters for maximum sequences and output directory
MAX_SEQUENCES = 5000  # Set the maximum number of sequences to keep
OUTPUT_DIR = truncated_a3m_output  # Define the output directory for trimmed files

# Define the directory containing .a3m files
a3m_dir = sorted_dir

def trim_a3m_file(a3m_file):
    """
    Trims an .a3m file to a maximum number of sequences and saves it in the OUTPUT_DIR.
    
    Parameters:
    - a3m_file: Path to the original .a3m file.
    """
    # Ensure the output directory exists
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        
    # Define the output file path
    filename = os.path.basename(a3m_file)
    trimmed_a3m_file = os.path.join(OUTPUT_DIR, filename)
    
    trimmed_lines = []
    count = 0
    
    with open(a3m_file, "r") as file:
        for line in file:
            # Add headers and sequence pairs until MAX_SEQUENCES is reached
            if line.startswith(">"):
                if count >= MAX_SEQUENCES:
                    break
                trimmed_lines.append(line)
                count += 1
            else:
                trimmed_lines.append(line)
    
    # Write the trimmed sequences to the output file
    with open(trimmed_a3m_file, "w") as file:
        file.writelines(trimmed_lines)
    
    print(f"Trimmed {a3m_file} to {MAX_SEQUENCES} sequences in {trimmed_a3m_file}")

def process_a3m_files(a3m_dir):
    """
    Trims all .a3m files in a directory to a maximum number of sequences and saves them in the OUTPUT_DIR.
    
    Parameters:
    - a3m_dir: Directory containing the .a3m files.
    """
    a3m_files = [os.path.join(a3m_dir, filename) for filename in os.listdir(a3m_dir) if filename.endswith(".a3m")]
    
    for a3m_file in a3m_files:
        trim_a3m_file(a3m_file)

# Trim all .a3m files in the directory
process_a3m_files(a3m_dir)


Trimmed /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/sorted_a3m/NP_001033043.1_pos_198.a3m to 5000 sequences in /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/trunc_a3m/NP_001033043.1_pos_198.a3m
Trimmed /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/sorted_a3m/XP001_pos_001.a3m to 5000 sequences in /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/trunc_a3m/XP001_pos_001.a3m
Trimmed /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/sorted_a3m/NP_002211.1_pos_31.a3m to 5000 sequences in /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6a1c1a979/PhD/Pulldown/Preliminary/Original_Jackhmmer/Run/Output/trunc_a3m/NP_002211.1_pos_31.a3m
Trimmed /media/pythagoras/895c7a01-1c19-48a4-9803-d0d6

#### 10. Combines the .a3m file of the bait and the prey proteins and creates the paired MSAs which serve as input for ColabFold

In [20]:
import os
import multiprocessing


# Constants for file paths
INPUT_DIR = truncated_a3m_output
Bait_FILE = os.path.join(INPUT_DIR, "bait_sequence.a3m")
COMBINED_OUTPUT_DIR = combined_dir


def read_a3m(file_path):
    """Reads an .a3m file and returns up to max_sequences headers and sequences."""
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    headers, sequences, seq_dict = [], [], {}
    current_header, count = None, 0  # Initialize variables

    for line in lines:
        if line.startswith('>'):
            current_header = line.strip()
            headers.append(current_header)
            seq_dict[current_header] = []
            count += 1
        elif current_header:
            seq_dict[current_header].append(line.strip())

    for header in headers:
        sequences.append((header, ''.join(seq_dict[header])))

    return sequences


def write_combined_a3m(file1_sequences, file2_sequences, output_file):
    """Combines two .a3m files with appropriate padding and custom headers."""
    with open(output_file, 'w') as file:
        len1 = len(file1_sequences[0][1])
        len2 = len(file2_sequences[0][1])
        
        # Combined header
        file.write(f"#{len1},{len2}\t1,1\n")

        # Write combined 101 and 102 sequence
        file.write(f">101\t102\n")
        combined_sequence = file1_sequences[0][1] + file2_sequences[0][1]
        file.write(combined_sequence + '\n')
        
        # Write padded 101 sequence
        file.write(">101\n")
        file.write(file1_sequences[0][1] + '-' * len2 + '\n')

        # Write other sequences from file1
        for header, seq in file1_sequences:
            file.write(f"{header}_1\n")
            file.write(seq.ljust(len1 + len2, '-') + '\n')

        # Write padded 102 sequence
        file.write(">102\n")
        file.write('-' * len1 + file2_sequences[0][1] + '\n')

        # Write other sequences from file2
        for header, seq in file2_sequences:
            file.write(f"{header}_2\n")
            file.write('-' * len1 + seq + '\n')


def process_single_file(file_name, bait_sequences):
    """Processes a single .a3m file by combining it with the actin sequences."""
    input_file_path = os.path.join(INPUT_DIR, file_name)
    output_file_path = os.path.join(COMBINED_OUTPUT_DIR, f"c_{file_name}")

    input_sequences = read_a3m(input_file_path)
    write_combined_a3m(input_sequences, bait_sequences, output_file_path)
    print(f"Combined {file_name} with actin.a3m into {os.path.basename(output_file_path)}")


def main():
    # Ensure output directory exists
    os.makedirs(COMBINED_OUTPUT_DIR, exist_ok=True)

    # Read actin sequences once
    bait_sequences = read_a3m(Bait_FILE)

    # List all input files
    file_names = [f for f in os.listdir(INPUT_DIR) if f.endswith(".a3m")]

    # Multiprocessing to process files in parallel
    num_cores = multiprocessing.cpu_count()  # Adjust based on system
    with multiprocessing.Pool(processes=num_cores) as pool:
        pool.starmap(process_single_file, [(file_name, bait_sequences) for file_name in file_names])


if __name__ == "__main__":
    main()


Combined NP_001033043.1_pos_198.a3m with actin.a3m into c_NP_001033043.1_pos_198.a3m
Combined XP001_pos_001.a3m with actin.a3m into c_XP001_pos_001.a3m
Combined NP_002211.1_pos_31.a3m with actin.a3m into c_NP_002211.1_pos_31.a3m
Combined NP_001277260.1_pos_748.a3m with actin.a3m into c_NP_001277260.1_pos_748.a3m
Combined NP_001239264.1_pos_353.a3m with actin.a3m into c_NP_001239264.1_pos_353.a3m
Combined NP_001307366.1_pos_1072.a3m with actin.a3m into c_NP_001307366.1_pos_1072.a3mCombined NP_001159632.1_pos_202.a3m with actin.a3m into c_NP_001159632.1_pos_202.a3m

Combined NP_065821.1_pos_178.a3m with actin.a3m into c_NP_065821.1_pos_178.a3mCombined NP_001291410.1_pos_105.a3m with actin.a3m into c_NP_001291410.1_pos_105.a3mCombined XP002_pos_001.a3m with actin.a3m into c_XP002_pos_001.a3m


Combined XP003_pos_001.a3m with actin.a3m into c_XP003_pos_001.a3mCombined NP_001375333.1_pos_143.a3m with actin.a3m into c_NP_001375333.1_pos_143.a3mCombined NP_001128668.1_pos_143.a3m with actin.a